In [52]:
import shutil, os, csv, itertools, glob

import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

from sklearn.metrics import confusion_matrix

import pandas as pd
import pickle as pk

cuda = torch.cuda.is_available()

import os
import time
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import collections
from sklearn.model_selection import StratifiedKFold
%matplotlib inline
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
import matplotlib.pyplot as plt
import torchvision.utils
import numpy as np
import random

import torch
from torch.autograd import Variable   
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D

import copy

In [53]:
#####################################################################################################################
#  Classifiers: Code directly modified based on the PyTorch Model Zoo Implementations
#####################################################################################################################

class CNN_LSTM(nn.Module):

    def __init__(self, in_dim, em_dim, hidden_dim, num_layers, dropout, bidirectional, out_dim, batch_size):
        super(CNN_LSTM, self).__init__()
        self.arch = 'cnn_lstm'
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_dir = 2 if bidirectional else 1
        self.num_layers = num_layers
        
        self.cnn = nn.Linear(in_dim,em_dim) # from input dimension to embedding dim
                
        self.lstm = nn.LSTM(
                input_size=em_dim,
                hidden_size=hidden_dim,
                num_layers=num_layers,
               # dropout=dropout,
                bidirectional=bidirectional
            )


        self.hidden2pred = nn.Sequential(
            nn.Linear(hidden_dim, 2*out_dim),
            nn.ReLU(True),
            nn.Dropout(dropout),
            
            nn.Linear(2*out_dim, out_dim),
            nn.ReLU(True),
            nn.Dropout(dropout),
            nn.Linear(out_dim, out_dim)
        )


    def forward(self, x, batch_size = None): # x is (batch_size, seq_len, input_dim), permute to (seq_len, batch_size, input_dim)

        x_perm = x.permute(1, 0, 2)
        x_perm = self.cnn(x_perm)
        # See: https://discuss.pytorch.org/t/solved-why-we-need-to-detach-variable-which-contains-hidden-representation/1426/2
        lstm_out, (h, c) = self.lstm(x_perm)
        
        #print (h.shape)
        h = h.squeeze(0)
        y = self.hidden2pred(h) 

        return y



In [54]:
#####################################################################################################################
#  DataLoaders
#####################################################################################################################

class LSTM_dataset(Dataset):

    def __init__(self, X, y, train = True):
        self.X = X
        self.y = y
        self.train = train

        if self.train:
            self.train_labels = self.y
            self.train_data = self.X

        else:
            self.test_labels = self.y
            self.test_data = self.X


    def __getitem__(self, index):
        
        if self.train:
            input_x, output_y = self.train_data[index], self.train_labels[index]
        else:
            input_x, output_y = self.test_data[index], self.test_labels[index]

        return input_x, output_y

    def __len__(self):
        return len(self.X)

In [55]:
seq_len = 14 # past 7 days for example
pre_len = 1 # predict future 1 day, for example

In [56]:
data = pd.read_csv('../../data/logistics/ccc_log.csv')

X = []
y = []

i = seq_len

while i < 700: # rest few are 0s
    if i + pre_len <= 700:
        X.append(data.iloc[i - seq_len:i, 1:4].values)
        y.append(data.iloc[i:i+pre_len, 1:4].values)
        
    i = i + 1
    



In [57]:
def rmse(preds, real_y):
    res = torch.pow(preds - real_y, 2)
    #print (res.shape)
    return torch.sqrt(torch.sum(res)/(res.shape[0]*res.shape[1]*res.shape[2]))

def mse_only(preds, real_y):
    res = torch.pow(preds - real_y, 2)
    #print (res.shape)
    return torch.sum(res)

In [70]:
# split to training and testing

X_train = X[0:623]
y_train = y[0:623]

X_test = X[623:]
y_test = y[623:]

batch_size = 5 # reduce can increase the performance

In [71]:
train_dataset = LSTM_dataset(X_train, y_train, True)
test_dataset = LSTM_dataset(X_test, y_test, False)

train_loader = DataLoader(train_dataset, batch_size=batch_size,shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0) 

In [72]:
model = CNN_LSTM(
            in_dim=3,
            em_dim = 10,
            hidden_dim=20, #input_size, #this is werid, because we want to calculate the cosine similarity
            num_layers=1,
            dropout=0.2,
            bidirectional=False,
            out_dim=3*pre_len,
            batch_size=batch_size
        )

In [74]:
train_epochs = 20



if cuda:
    print ('use cuda!!!!!!!!!!!!!!')
    model.cuda()
optimizer = optim.Adam(model.parameters(), lr = 0.001, amsgrad=True)

for epoch in range(train_epochs):
    train_loss = 0
    model.train()
    for batch_idx, (data_x, real_y) in enumerate(train_loader):
        if (len(data_x) < batch_size):# not enough to form a batch
            continue
        data_x, real_y = Variable(data_x.float()), Variable(real_y.float())
            
        if cuda:
            data_x, real_y = data_x.cuda(), real_y.cuda()
            
        preds = model(data_x)
        l = rmse(preds, real_y)
        l1 = mse_only(preds, real_y)
        
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        
        train_loss += l1
        
        #if batch_idx % 10 == 0: #batch index
        #    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        #    epoch, batch_idx * len(data_x), len(train_loader.dataset),
        #    100. * batch_idx / len(train_loader), l))
    
    #print (batch_idx)
    #print (len(data_x))
    train_loss /= batch_idx * batch_size * 3
    train_loss = torch.sqrt(train_loss)
        
    print ('Train Epoch: {}, Total RMSE loss: {}'.format(epoch, train_loss))
        

use cuda!!!!!!!!!!!!!!
Train Epoch: 0, Total RMSE loss: 0.5141170024871826
Train Epoch: 1, Total RMSE loss: 0.5206380486488342
Train Epoch: 2, Total RMSE loss: 0.5146805644035339
Train Epoch: 3, Total RMSE loss: 0.520697295665741
Train Epoch: 4, Total RMSE loss: 0.5164098143577576
Train Epoch: 5, Total RMSE loss: 0.5197269916534424
Train Epoch: 6, Total RMSE loss: 0.5195407271385193
Train Epoch: 7, Total RMSE loss: 0.5221380591392517
Train Epoch: 8, Total RMSE loss: 0.5222545266151428
Train Epoch: 9, Total RMSE loss: 0.5195395350456238
Train Epoch: 10, Total RMSE loss: 0.517933189868927
Train Epoch: 11, Total RMSE loss: 0.5191039443016052
Train Epoch: 12, Total RMSE loss: 0.5205318331718445
Train Epoch: 13, Total RMSE loss: 0.5210170149803162
Train Epoch: 14, Total RMSE loss: 0.5208792090415955
Train Epoch: 15, Total RMSE loss: 0.5218291878700256
Train Epoch: 16, Total RMSE loss: 0.5207980871200562
Train Epoch: 17, Total RMSE loss: 0.5207440257072449
Train Epoch: 18, Total RMSE loss: 0

In [13]:
model.train()
mu = [] # mu and std
std = [] 
sample_size = 1000
all_preds = []

batch_size = 1

for batch_idx, (data_x, real_y) in enumerate(test_loader):
    

    data_x, real_y = Variable(data_x.float()), Variable(real_y.float())

    if cuda:
        data_x, real_y = data_x.cuda(), real_y.cuda()
    
    #print (len(data_x))
    tem = []
    for i in range(sample_size): # run 1000 times for each observation
        #print ('here')
        preds = model(data_x)
        tem.append(preds.detach().cpu().numpy()[0])
        
    all_preds.append(tem)
        
    mu.append(np.mean(tem, axis = 0))
    std.append(np.std(tem, axis = 0))

In [14]:
mu

[array([0.5313928 , 0.5642335 , 0.39894202], dtype=float32),
 array([0.5328076 , 0.5680059 , 0.40283632], dtype=float32),
 array([0.53728276, 0.57675004, 0.40937272], dtype=float32),
 array([0.5267989 , 0.55529934, 0.39230782], dtype=float32),
 array([0.53563344, 0.5726913 , 0.40545055], dtype=float32),
 array([0.5286194 , 0.55851346, 0.39435595], dtype=float32),
 array([0.53353727, 0.56915116, 0.40340003], dtype=float32),
 array([0.5292436 , 0.55997545, 0.39570382], dtype=float32),
 array([0.52054673, 0.54246336, 0.38206074], dtype=float32),
 array([0.5191592 , 0.5393974 , 0.37939182], dtype=float32),
 array([0.5298652 , 0.5615551 , 0.39727512], dtype=float32),
 array([0.5195389 , 0.5396299 , 0.37902147], dtype=float32),
 array([0.5189204 , 0.53805685, 0.3774586 ], dtype=float32),
 array([0.5169565, 0.5350243, 0.3760461], dtype=float32),
 array([0.5199276 , 0.5404587 , 0.37972248], dtype=float32),
 array([0.5175194 , 0.53564554, 0.37600878], dtype=float32),
 array([0.5256912 , 0.55251

In [15]:
std

[array([0.12927662, 0.26515573, 0.21227191], dtype=float32),
 array([0.12558945, 0.2563207 , 0.20432268], dtype=float32),
 array([0.12311675, 0.25222522, 0.20182723], dtype=float32),
 array([0.12659846, 0.25891986, 0.20679896], dtype=float32),
 array([0.1204059 , 0.24649699, 0.1968955 ], dtype=float32),
 array([0.12551518, 0.2575867 , 0.20645495], dtype=float32),
 array([0.12296677, 0.2514991 , 0.20088047], dtype=float32),
 array([0.12494023, 0.25610587, 0.2050004 ], dtype=float32),
 array([0.12645891, 0.2589852 , 0.20718248], dtype=float32),
 array([0.12256359, 0.2507031 , 0.20020753], dtype=float32),
 array([0.12334801, 0.25260472, 0.20214362], dtype=float32),
 array([0.12222566, 0.25097236, 0.20131798], dtype=float32),
 array([0.11594684, 0.23738939, 0.18967402], dtype=float32),
 array([0.11826085, 0.24164161, 0.19282402], dtype=float32),
 array([0.11953995, 0.24490002, 0.1957979 ], dtype=float32),
 array([0.1197371 , 0.24531098, 0.19624232], dtype=float32),
 array([0.12217442, 0.25

In [16]:
mu = pd.DataFrame(mu)
mu.columns = ['mu1', 'mu2', 'mu3']
mu.to_csv('mu_test_dataset.csv')

In [17]:
std = pd.DataFrame(std)
std.columns = ['std1', 'std2', 'std3']
std.to_csv('std_test_dataset.csv')

In [63]:
model.eval()

all_preds = []

batch_size = 1

for batch_idx, (data_x, real_y) in enumerate(test_loader):
    

    data_x, real_y = Variable(data_x.float()), Variable(real_y.float())

    if cuda:
        data_x, real_y = data_x.cuda(), real_y.cuda()
    

    preds = model(data_x)
        
    all_preds.append(preds.detach().cpu().numpy()[0])
        


In [64]:
all_preds

[array([0.600474  , 0.7123224 , 0.50865614], dtype=float32),
 array([0.6010091 , 0.71292555, 0.50912213], dtype=float32),
 array([0.6433847 , 0.75926095, 0.55348974], dtype=float32),
 array([0.6634928 , 0.7813034 , 0.57425404], dtype=float32),
 array([0.6045804 , 0.71668756, 0.51360846], dtype=float32),
 array([0.5795986 , 0.68904203, 0.48917305], dtype=float32),
 array([0.5947572, 0.7059196, 0.5034635], dtype=float32),
 array([0.60655737, 0.71896386, 0.5150799 ], dtype=float32),
 array([0.5860251 , 0.6962472 , 0.49497068], dtype=float32),
 array([0.6083877 , 0.7210045 , 0.51679105], dtype=float32),
 array([0.60739815, 0.71992147, 0.5157601 ], dtype=float32),
 array([0.6050929 , 0.71739805, 0.51336086], dtype=float32),
 array([0.6040031, 0.7162075, 0.5122138], dtype=float32),
 array([0.60430115, 0.7165407 , 0.51248837], dtype=float32),
 array([0.6082202 , 0.72084427, 0.5164956 ], dtype=float32),
 array([0.64046186, 0.7561212 , 0.5501357 ], dtype=float32),
 array([0.65728015, 0.77453434

In [20]:
all_preds = pd.DataFrame(all_preds)
all_preds.columns = ['point_prediction1', 'point_prediction2', 'point_prediction3']
all_preds.to_csv('point_prediction.csv')